In [ ]:
!pip install vantage6-client

In [ ]:
vantage_server = ''
vantage_user = ''
vantage_password = ''

In [ ]:
# Setup, run once
from vantage6.client import Client

In [ ]:
client = Client(vantage_server, 443, "/api")
client.authenticate(vantage_user, vantage_password)
client.setup_encryption(None)

In [ ]:
# Find out the organizations and collaborations we can run on
import json
collaboration_list = client.collaboration.list()
collaboration_index = 0
organization_ids_ = [ ]

for organization in collaboration_list[collaboration_index]['organizations']:
    organization_ids_.append(organization['id'])

In [ ]:
# Write your query here
query = """
prefix snomedct: <http://purl.bioontology.org/ontology/SNOMEDCT/>
prefix roo: <http://www.cancerdata.org/roo/>
prefix ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?patientIdentifier ?age ?sex ?diseaseDate ?location ?tumourType
WHERE {
    ?patient roo:P100018 ?sexClass . #has biological sex
    ?sexClass roo:P100042 ?sexValue .
    BIND( xsd:string(IF(?sexValue = 2, "female", "male")) AS ?sex) .
}
"""

In [ ]:
# Request the structure to be built
input_ = {
    "master": "true",
    "method":"master", 
    "args": [ ],
    "kwargs": {
        "query": query
    }
}

task = client.post_task(
    name="ExploreData",
    image="jaspersnel/v6-summary-py-sparql:swat4ls-0.1.0",
    collaboration_id=collaboration_list[collaboration_index]['id'],#Get the first collaboration associated with user
    input_= input_,
    organization_ids=[organization_ids_[0]]
)

print(json.dumps(task, indent=2))

In [ ]:
# Wait for results
import time
import json
resultObjRef = task.get("results")[0]
resultObj = client.result.get(resultObjRef['id'])
attempts = 1
while((resultObj["finished_at"] == None) and attempts < 10):
    print("waiting...")
    time.sleep(5)
    resultObj = client.result.get(resultObjRef['id'])
    attempts += 1
results = resultObj['result']

print(results)